# Eat Safe, Love

## Notebook Set Up

In [27]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [28]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [29]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [30]:
# review the collections in our database
collections = db.list_collection_names()
pprint(collections)

['establishments']


In [31]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [32]:
# Find the establishments with a hygiene score of 20
query = {"scores.Hygiene": 20}

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print(f"{count} establishments have hygiene scores of 20\n")

# Display the first document in the results using pprint
first = establishments.find_one(query)
pprint(first)

41 establishments have hygiene scores of 20

{'AddressLine1': '5-6 Southfields Road',
 'AddressLine2': 'Eastbourne',
 'AddressLine3': 'East Sussex',
 'AddressLine4': '',
 'BusinessName': 'The Chase Rest Home',
 'BusinessType': 'Caring Premises',
 'BusinessTypeID': 5,
 'ChangesByServerID': 0,
 'Distance': 4613.888288172291,
 'FHRSID': 110681,
 'LocalAuthorityBusinessID': '4029',
 'LocalAuthorityCode': '102',
 'LocalAuthorityEmailAddress': 'Customerfirst@eastbourne.gov.uk',
 'LocalAuthorityName': 'Eastbourne',
 'LocalAuthorityWebSite': 'http://www.eastbourne.gov.uk/foodratings',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'BN21 1BU',
 'RatingDate': '2021-09-23T00:00:00',
 'RatingKey': 'fhrs_0_en-gb',
 'RatingValue': {'$toInt': '$RatingValue'},
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('673ce541ae3a6729175b271c'),
 'geocode': {'latitude': {'$toDouble': '$geocode.latitude'},
             'longitude': {'$toDouble': '$geocode.longitude'}},
 'links': [{'href': 'h

In [33]:
# Convert the result to a Pandas DataFrame
results = list(establishments.find(query))
df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print(f"Number of rows: {len(df)}")

# Display the first 10 rows of the DataFrame
print(df.head(10))


Number of rows: 41
                        _id   FHRSID  ChangesByServerID  \
0  673ce541ae3a6729175b271c   110681                  0   
1  673ce541ae3a6729175b2a9d   612039                  0   
2  673ce542ae3a6729175b2da7   730933                  0   
3  673ce542ae3a6729175b2f95   172735                  0   
4  673ce542ae3a6729175b2fa4   172953                  0   
5  673ce542ae3a6729175b3943   512854                  0   
6  673ce542ae3a6729175b3b64  1537089                  0   
7  673ce544ae3a6729175b508f   155648                  0   
8  673ce544ae3a6729175b54d3  1012883                  0   
9  673ce545ae3a6729175b5ce5   644109                  0   

  LocalAuthorityBusinessID               BusinessName  \
0                     4029        The Chase Rest Home   
1                1970/FOOD                 Brenalwood   
2                1698/FOOD              Melrose Hotel   
3             PI/000023858              Seaford Pizza   
4             PI/000024532              Golden

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [34]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {
    "LocalAuthorityName": "London", 
    "RatingValue": {"$gte": 4}
}

# Use count_documents to display the number of documents in the result
num = establishments.count_documents(query)
print(f"Number of documents: {num}\n")

# Display the first document in the results using pprint
first = establishments.find_one(query)
print(f"First document: {first}")

Number of documents: 0

First document: None


In [35]:
# Convert the result to a Pandas DataFrame
results = list(establishments.find(query))
df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print(f"Number of rows: {len(df)}\n")

# Display the first 10 rows of the DataFrame
print("First 10 rows:")
print(df.head(10))

Number of rows: 0

First 10 rows:
Empty DataFrame
Columns: []
Index: []


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [37]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 0.08384000
longitude = 51.49014200

query = {
    "RatingValue": 5,
    "geocode.latitude": {"$gte": latitude - degree_search, "$lte": latitude + degree_search},
    "geocode.longitude": {"$gte": longitude - degree_search, "$lte": longitude + degree_search}
}
sort = [("scores.Hygiene", -1)]  # -1 for descending order
limit = 100

# Print the results
results = list(establishments.find(query).sort(sort).limit(limit))
print(f"Found {len(results)} establishments with RatingValue 5 within {degree_search} degrees of the given latitude and longitude:")
pprint(results)

Found 0 establishments with RatingValue 5 within 0.01 degrees of the given latitude and longitude:
[]


In [38]:
# Convert result to Pandas DataFrame
df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print(f"Number of rows: {len(df)}\n")

# Display the first 10 rows of the DataFrame
print("First 10 rows:")
print(df.head(10))

Number of rows: 0

First 10 rows:
Empty DataFrame
Columns: []
Index: []


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [40]:
# Create a pipeline that:
pipeline = [
    # 1. Matches establishments with a hygiene score of 0
    {"$match": {"scores.Hygiene": 0}},
    # 2. Groups the matches by Local Authority
    {"$group": {
        "_id": "$LocalAuthorityName",
        "count": {"$sum": 1}
    }},
    # 3. Sorts the matches from highest to lowest
    {"$sort": {"count": -1}}  # -1 for descending order
]
results = list(establishments.aggregate(pipeline))

# Print the number of documents in the result
print(f"{len(results)} Local Authorities with hygiene score 0.\n")

# Print the first 10 results
print("First 10 results:")
pprint(results[:10])

55 Local Authorities with hygiene score 0.

First 10 results:
[{'_id': 'Thanet', 'count': 1130},
 {'_id': 'Greenwich', 'count': 882},
 {'_id': 'Maidstone', 'count': 713},
 {'_id': 'Newham', 'count': 711},
 {'_id': 'Swale', 'count': 686},
 {'_id': 'Chelmsford', 'count': 680},
 {'_id': 'Medway', 'count': 672},
 {'_id': 'Bexley', 'count': 607},
 {'_id': 'Southend-On-Sea', 'count': 586},
 {'_id': 'Tendring', 'count': 542}]


In [41]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {len(df)}\n")

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df.head(10))

Number of rows in the DataFrame: 55

First 10 rows of the DataFrame:
               _id  count
0           Thanet   1130
1        Greenwich    882
2        Maidstone    713
3           Newham    711
4            Swale    686
5       Chelmsford    680
6           Medway    672
7           Bexley    607
8  Southend-On-Sea    586
9         Tendring    542
